In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html

/usr/local/python/3.10.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1+cu117


In [3]:
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html

In [7]:
#pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

Note: you may need to restart the kernel to use updated packages.


In [11]:
import torch_geometric
from torch_geometric.datasets import Planetoid, TUDataset
from torch_geometric.loader import DataLoader

In [3]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [4]:
len(dataset)

1

In [5]:
data = dataset[0]

In [6]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [9]:
data.y.sum().item()

7781

In [46]:
#help(DataLoader)

In [14]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!


In [45]:
#help(batch)

In [41]:
for batch in loader:
    batch

In [42]:
batch

DataBatch(edge_index=[2, 2738], x=[813, 21], y=[24], batch=[813], ptr=[25])

In [43]:
 batch.num_nodes

813

In [44]:
batch.num_graphs

24

In [34]:
from torch_geometric.utils import scatter

In [35]:
x = scatter(batch.x, batch.batch, dim=0, reduce='mean')
x.size()

torch.Size([24, 21])

In [47]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [48]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [49]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [52]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [53]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [54]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8060
